## 基于yolov5的数独检测
### 0.环境安装

In [10]:
% pip install -qr requirements.txt
! git clone https://github.com/ultralytics/yolov5.git

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


You should consider upgrading via the 'C:\Python39\python.exe -m pip install --upgrade pip' command.


created virtual environment CPython3.9.11.final.0-64 in 1026ms
  creator CPython3Windows(dest=C:\Users\pk5ls\OneDrive\Interest\Pending\sudoku\sudoku, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=C:\Users\pk5ls\AppData\Local\pypa\virtualenv)
    added seed packages: pip==23.0.1, setuptools==67.6.1, wheel==0.40.0
  activators BashActivator,BatchActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


You should consider upgrading via the 'C:\Python39\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/36/98/fab8d982e2e2b57bdebcad64c7e5b5a14ac91c657cac509b9cf3fbea49d2/opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.7.0.68
    Uninstalling opencv-python-4.7.0.68:
      Successfully uninstalled opencv-python-4.7.0.68


You should consider upgrading via the 'C:\Python39\python.exe -m pip install --upgrade pip' command.
Cloning into 'yolov5'...


### 1.提取数独

In [13]:
import os
import torch
import datetime
import cv2
model = torch.hub.load('ultralytics/yolov5', 'custom', path='detect_sudoku.pt')
input_path = 'sudoku_pic'
output_path = 'sudoku_pic/extract'
timex = lambda :datetime.datetime.now()
for file_name in os.listdir(input_path):
    if file_name.endswith('.jpg') or file_name.endswith('.png'):
        print(f"正在处理{file_name}", end='')
        img = cv2.imread(os.path.join(input_path, file_name))
        # 使用YOLOv5检测
        results = model(img)
        # 得到置信度最高的数独检测结果
        sudoku_detection = None
        for result in results.pred[0]:
            if result[-1] == 0 and result[-2] > 0.8:
                sudoku_detection = result
                break
        # 提取数独
        if sudoku_detection is not None:
            xmin, ymin, xmax, ymax, confidence = sudoku_detection[:5]
            sudoku = img[int(ymin):int(ymax), int(xmin):int(xmax)]
            cv2.imwrite(os.path.join(output_path, f"extract_{file_name}"), sudoku)
            print(f"...提取成功！")
        else:
            print(f"...{file_name}未检测到数独！")
print(f"Done on {timex()}")

AttributeError: partially initialized module 'cv2' has no attribute 'gapi_wip_gst_GStreamerPipeline' (most likely due to a circular import)

### 2.提取数独中的数字

In [ ]:
# Iterate over images in input folder
fp = list(os.listdir(output_path))
for file_name in fp:
    if file_name.split('_')[0] != 'extract':
        continue
    # Load image
    img = cv2.imread(os.path.join(output_path, file_name))
    # Get dimensions of image
    height, width, _ = img.shape
    # Calculate size of each small image
    size = int(height / 9)
    # Iterate over rows and columns of small images
    for row in range(9):
        for col in range(9):
            # Calculate coordinates of small image
            x1 = col * size
            y1 = row * size
            x2 = x1 + size
            y2 = y1 + size
            # Crop small image from main image
            small_img = img[y1:y2, x1:x2]
            # Save small image to output folder
            small_img_file_name = '{}_{}.png'.format(os.path.splitext(file_name)[0], row * 9 + col)
            cv2.imwrite(os.path.join(output_path, small_img_file_name), small_img)
print(f"Done on {timex()}")

### 3.单个数字图片预处理
在识别单个数字之前，需要对图片进行预处理。
受限于训练模型，进行二值化+去黑线的预处理可以大幅度提高识别准确率

In [ ]:
from PIL import Image, ImageOps
import os

# 读取图片并转换为黑白图
folder_path = 'sudoku_pic/extract'
for filename in os.listdir(folder_path):
    if len(filename.split('_')) != 3:
        continue
    img = Image.open(f"{folder_path}/{filename}").convert('L')
    # 获取图片的宽度和高度
    width, height = img.size
    cl = []
    # 遍历每一行，如果整行像素点>=80%部分不是白色，则将该行像素点全部转换为白色
    for y in range(height):
        pixels = [img.getpixel((x, y)) for x in range(width)]
        white_pixels = sum(1 for pixel in pixels if pixel == 255)
        if white_pixels < width*0.2:
            for x in range(width):
                cl.append((x, y))
    # 遍历每一列，如果整列像素点>=80%部分不是白色，则将该列像素点全部转换为白色
    for x in range(width):
        pixels = [img.getpixel((x, y)) for y in range(height)]
        white_pixels = sum(1 for pixel in pixels if pixel == 255)
        if white_pixels < height*0.2:
            for y in range(height):
                cl.append((x, y))
    # 将所有的白色像素点转换为黑色
    for x, y in cl:
        img.putpixel((x, y), 255)
    # 反转图片颜色
    img = ImageOps.invert(img)
    img.save(f"{folder_path}/ok/ok_{filename}")
print(f"Done on {timex()}")

### 4.识别数字并转化为数独


In [ ]:
from torchvision.transforms import ToTensor, Resize
from pathlib import Path
import sys
from models.experimental import attempt_load
from utils.general import non_max_suppression
from utils.torch_utils import select_device
import numpy as np
from PIL import Image

sys.path.insert(0, str(Path('yolov5')))

def load_model(model_path):
    device = select_device()
    model = attempt_load(model_path, device)
    model.eval()
    return model

def predict(model, image_path):
    device = select_device()
    img = Image.open(image_path).convert("RGB")
    img = Resize((128, 128))(img)
    img_tensor = ToTensor()(img).unsqueeze(0).to(device)
    pred = model(img_tensor)[0]
    # Apply non-max suppression
    results = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False)
    return results, img.size

def process_images(model_path, image_folder):
    model = load_model(model_path)
    sudoku = np.zeros((9, 9), dtype=np.int32)
    for img_name in sorted(os.listdir(image_folder)):
        if img_name.startswith("ok_extract_"):
            row = int(img_name.split("_")[3].split('.')[0]) // 9
            col = int(img_name.split("_")[3].split('.')[0]) % 9
            img_path = os.path.join(image_folder, img_name)
            results, img_size = predict(model, img_path)
            if len(results) > 0 and len(results[0]) > 0:
                most_likely_class = int(results[0][0][5].item())
                sudoku[row, col] = most_likely_class
        else:
            continue
    print(sudoku)
process_images("detect_number.pt", "sudoku_pic/extract/ok")
print(f"Done on {timex()}")